In [44]:
from pyspark.sql import SparkSession

spark= (SparkSession.builder.appName("Capstone")\
       .config("hive.metastore.uris","thrift://ip-10-1-2-24.ap-south-1.compute.internal:9083")
       .enableHiveSupport().getOrCreate())

In [45]:
spark

In [46]:
from pyspark.sql import functions as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from pyspark.sql.window import Window
# from pyspark.sql.functions import * #lot of functions available here
from pyspark.sql.types import IntegerType, StringType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.sql.types import *
from pyspark.sql.functions import when, count

1. A list showing employee number, last name, first name, sex, and salary for each employee

In [47]:
spark.sql("""select emp123.emp_no, emp123.last_name, emp123.first_name, 
emp123.sex,
         Salary.salary from emp123 join Salary on emp123.emp_no == Salary.emp_no""").show(5,truncate=False)

+------+---------+----------+---+------+
|emp_no|last_name|first_name|sex|salary|
+------+---------+----------+---+------+
|473302|Zallocco |Hideyuki  |M  |40000 |
|475053|Delgrande|Byong     |F  |53422 |
|57444 |Babb     |Berry     |F  |48973 |
|421786|Verhoeff |Xiong     |M  |40000 |
|282238|Baumann  |Abdelkader|F  |40000 |
+------+---------+----------+---+------+
only showing top 5 rows



2. A list showing first name, last name, and hire date for employees who were hired in 1986.

In [48]:
spark.sql("""select emp123.first_name, emp123.last_name,emp123.hire_date from emp123 
          where hire_date BETWEEN '1/1/1986' AND '12/31/1986' """).show(20,truncate=False)

+----------+-----------+----------+
|first_name|last_name  |hire_date |
+----------+-----------+----------+
|Xiong     |Verhoeff   |11/26/1987|
|Abdelkader|Baumann    |1/18/1991 |
|Eran      |Cusworth   |11/14/1986|
|Xudong    |Samarati   |11/13/1985|
|Lihong    |Magliocco  |10/23/1993|
|Shuichi   |Tyugu      |1/17/1995 |
|Bojan     |Zallocco   |10/14/1986|
|Bilhanan  |Wuwongse   |10/6/1993 |
|Elliott   |Perl       |10/29/1987|
|Nechama   |Copas      |10/27/1987|
|Pantung   |Cools      |1/28/1994 |
|Hairong   |Schaar     |12/24/1987|
|Mohit     |Speek      |1/14/1986 |
|Bader     |Chinal     |1/8/1990  |
|Gad       |Nollmann   |10/1/1985 |
|Munehiko  |Janocha    |12/11/1988|
|Shaowen   |Krone      |11/18/1990|
|Rosalie   |Rousseau   |11/7/1998 |
|Hitomi    |Gunderson  |11/22/1987|
|Leni      |Pusterhofer|1/29/1995 |
+----------+-----------+----------+
only showing top 20 rows



3. A list showing the manager of each department with the following information: department number, department name, the manager's employee number, last name, first name.

In [49]:
spark.sql("""select depart.dept_no, depart.dept_name, emp123.emp_no,
         emp123.last_name,emp123.first_name from depart join
         depart_manager on depart.dept_no == depart_manager.dept_no
         join emp123 on depart_manager.emp_no == emp123.emp_no""").show(10,truncate=False)

+-------+--------------------+------+------------+-----------+
|dept_no|dept_name           |emp_no|last_name   |first_name |
+-------+--------------------+------+------------+-----------+
|d009   |"Customer Service"  |111877|Spinelli    |Xiaobin    |
|d008   |"Research"          |111534|Kambil      |Hilary     |
|d006   |"Quality Management"|110765|Hofmeyr     |Rutger     |
|d004   |"Production"        |110420|Ghazalie    |Oscar      |
|d006   |"Quality Management"|110725|Onuegbe     |Peternela  |
|d001   |"Marketing"         |110022|Markovitch  |Margareta  |
|d007   |"Sales"             |111035|Kaelbling   |Przemyslawa|
|d005   |"development"       |110511|Hagimont    |DeForest   |
|d005   |"development"       |110567|DasSarma    |Leon       |
|d003   |"Human Resources"   |110183|Ossenbruggen|Shirish    |
+-------+--------------------+------+------------+-----------+
only showing top 10 rows



4. A list showing the department of each employee with the following information: employee number, last name, first name, and department name.

In [50]:
spark.sql("""select depart.dept_name, emp123.emp_no,
         emp123.last_name,emp123.first_name from depart join
         depart_manager on depart.dept_no == depart_manager.dept_no
         join emp123 on depart_manager.emp_no == emp123.emp_no""").show(10,truncate=False)

+--------------------+------+------------+-----------+
|dept_name           |emp_no|last_name   |first_name |
+--------------------+------+------------+-----------+
|"Customer Service"  |111877|Spinelli    |Xiaobin    |
|"Research"          |111534|Kambil      |Hilary     |
|"Quality Management"|110765|Hofmeyr     |Rutger     |
|"Production"        |110420|Ghazalie    |Oscar      |
|"Quality Management"|110725|Onuegbe     |Peternela  |
|"Marketing"         |110022|Markovitch  |Margareta  |
|"Sales"             |111035|Kaelbling   |Przemyslawa|
|"development"       |110511|Hagimont    |DeForest   |
|"development"       |110567|DasSarma    |Leon       |
|"Human Resources"   |110183|Ossenbruggen|Shirish    |
+--------------------+------+------------+-----------+
only showing top 10 rows



5 A list showing first name, last name, and sex for employees whose first name is "Hercules" and last names begin with "B.“

In [51]:
spark.sql("""select emp123.first_name,emp123.last_name,emp123.sex
         from emp123 where first_name = 'Hercules' and last_name like 'B%'""").show(10,truncate=False)

+----------+-------------+---+
|first_name|last_name    |sex|
+----------+-------------+---+
|Hercules  |Baer         |M  |
|Hercules  |Biron        |F  |
|Hercules  |Birge        |F  |
|Hercules  |Berstel      |F  |
|Hercules  |Bernatsky    |M  |
|Hercules  |Bail         |F  |
|Hercules  |Bodoff       |M  |
|Hercules  |Benantar     |F  |
|Hercules  |Basagni      |M  |
|Hercules  |Bernardinello|F  |
+----------+-------------+---+
only showing top 10 rows



6. A list showing all employees in the Sales department, including their employee number, last name, first name, and department name.

In [52]:
spark.sql("""select emp123.last_name,emp123.first_name,depart.dept_name from dept_employee
    join emp123 on dept_employee.emp_no == emp123.emp_no join depart
    on dept_employee.dept_no == depart.dept_no where depart.dept_name = 'Sales'""").show(5, truncate=False)

+---------+----------+---------+
|last_name|first_name|dept_name|
+---------+----------+---------+
+---------+----------+---------+



7.A list showing all employees in the Sales and Development departments, including their employee number, last name, first name, and department name.

In [53]:
spark.sql("""select emp123.last_name,emp123.first_name,depart.dept_name from dept_employee
    join emp123 on dept_employee.emp_no == emp123.emp_no join depart
    on dept_employee.dept_no == depart.dept_no where depart.dept_name = 'Sales'
    or depart.dept_name= 'Development'""").show(5, truncate=False)

+---------+----------+---------+
|last_name|first_name|dept_name|
+---------+----------+---------+
+---------+----------+---------+



8.A list showing the frequency count of employee last names, in descending order. ( i.e., how many employees share each last name

In [54]:
x= spark.sql("select* from emp123")

In [55]:
x.groupBy('last_name').count().show()

+----------------+-----+
|       last_name|count|
+----------------+-----+
|       Delgrossi|  188|
|         Rullman|  171|
|        Hagimont|  164|
|         Lanteri|  173|
|          Nanard|  194|
|          Saoudi|  176|
|   Beutelspacher|  180|
|          Butner|  193|
|          Stille|  165|
|          Aamodt|  205|
|Sankaranarayanan|  183|
|         Kleiser|  193|
|       Besselaar|  188|
|         Solovay|  178|
|          Ritcey|  156|
|      Hinsberger|  187|
|            Lund|  170|
|     Shokrollahi|  179|
|      Taubenfeld|  200|
|            Gips|  167|
+----------------+-----+
only showing top 20 rows



12: 1

In [57]:
a= spark.sql("select * from emp123 join Salary on emp123.emp_no ==Salary.emp_no")

In [58]:
a.groupBy('sex').sum('salary').show()

+---+-----------+
|sex|sum(salary)|
+---+-----------+
|  F| 6357161017|
|  M| 9535330016|
+---+-----------+



12: 2

In [59]:
a.groupBy('left_').sum('salary').show()

+-----+-----------+
|left_|sum(salary)|
+-----+-----------+
|    0|14316882768|
|    1| 1575608265|
+-----+-----------+



12:3

In [60]:
b= spark.sql("select * from depart_manager join emp123 on depart_manager.emp_no == emp123.emp_no")

In [61]:
b.groupBy("dept_no").count().show()

+-------+-----+
|dept_no|count|
+-------+-----+
|   d005|    2|
|   d009|    4|
|   d003|    2|
|   d001|    2|
|   d007|    2|
|   d004|    4|
|   d002|    2|
|   d006|    4|
|   d008|    2|
+-------+-----+



12:4

In [62]:
c= spark.sql("select * from depart join dept_employee on depart.dept_no==dept_employee.dept_no")

In [63]:
c.groupBy('dept_name').count().show()

+--------------------+-----+
|           dept_name|count|
+--------------------+-----+
|"Quality Management"|20117|
|        "Production"|73485|
|       "development"|85707|
|          "Research"|21126|
|             "Sales"|52245|
|           "Finance"|17346|
|   "Human Resources"|17786|
|         "Marketing"|20211|
|  "Customer Service"|23580|
+--------------------+-----+



# A.Read the data from Hive tables

In [64]:
# Reading the data from hive table 

In [65]:

spark.sql("select * from depart").show()

+-------+--------------------+
|dept_no|           dept_name|
+-------+--------------------+
|   d001|         "Marketing"|
|   d002|           "Finance"|
|   d003|   "Human Resources"|
|   d004|        "Production"|
|   d005|       "development"|
|   d006|"Quality Management"|
|   d007|             "Sales"|
|   d008|          "Research"|
|   d009|  "Customer Service"|
+-------+--------------------+



In [66]:
spark.sql("select * from Salary").show()

+------+------+
|emp_no|salary|
+------+------+
| 10001| 60117|
| 10002| 65828|
| 10003| 40006|
| 10004| 40054|
| 10005| 78228|
| 10006| 40000|
| 10007| 56724|
| 10008| 46671|
| 10009| 60929|
| 10010| 72488|
| 10011| 42365|
| 10012| 40000|
| 10013| 40000|
| 10014| 46168|
| 10015| 40000|
| 10016| 70889|
| 10017| 71380|
| 10018| 55881|
| 10019| 44276|
| 10020| 40000|
+------+------+
only showing top 20 rows



In [67]:
spark.sql("select * from emp123").show()

+------+------------+----------+----------+----------+---+----------+--------------+-----------------------+-----+----------+
|emp_no|emp_title_id|birth_date|first_name| last_name|sex| hire_date|no_of_projects|last_performance_rating|left_| last_date|
+------+------------+----------+----------+----------+---+----------+--------------+-----------------------+-----+----------+
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
+------+------------+----------+----------+----------+---+----------+--------------+-----------------------+-----+----------+
only showing top 20 rows



In [68]:
spark.sql("select * from depart_manager").show()

+-------+------+
|dept_no|emp_no|
+-------+------+
|   d001|110022|
|   d001|110039|
|   d002|110085|
|   d002|110114|
|   d003|110183|
|   d003|110228|
|   d004|110303|
|   d004|110344|
|   d004|110386|
|   d004|110420|
|   d005|110511|
|   d005|110567|
|   d006|110725|
|   d006|110765|
|   d006|110800|
|   d006|110854|
|   d007|111035|
|   d007|111133|
|   d008|111400|
|   d008|111534|
+-------+------+
only showing top 20 rows



In [69]:
spark.sql("select * from dept_employee").show()

+------+-------+
|emp_no|dept_no|
+------+-------+
| 10001|   d005|
| 10002|   d007|
| 10003|   d004|
| 10004|   d004|
| 10005|   d003|
| 10006|   d005|
| 10007|   d008|
| 10008|   d005|
| 10009|   d006|
| 10010|   d004|
| 10010|   d006|
| 10011|   d009|
| 10012|   d005|
| 10013|   d003|
| 10014|   d005|
| 10015|   d008|
| 10016|   d007|
| 10017|   d001|
| 10018|   d004|
| 10018|   d005|
+------+-------+
only showing top 20 rows



In [70]:
spark.sql("select * from title").show()

+--------+------------------+
|title_id|             title|
+--------+------------------+
|   s0001|             Staff|
|   s0002|      Senior Staff|
|   e0001|Assistant Engineer|
|   e0002|          Engineer|
|   e0003|   Senior Engineer|
|   e0004|  Technique Leader|
|   m0001|           Manager|
+--------+------------------+



# B.You required to join all the tables at employee level

In [71]:
# Join all the tables 

In [72]:
df= spark.sql("""select emp123.emp_no, emp123.emp_title_id, emp123.birth_date, emp123.first_name,
    emp123.last_name, emp123.sex, emp123.hire_date, emp123.no_of_projects, emp123.last_performance_rating,
    emp123.left_,emp123.last_date, Salary.salary, title.title, dept_employee.dept_no,
    depart.dept_name from emp123 join dept_employee on emp123.emp_no == dept_employee.emp_no join title
    on emp123.emp_title_id == title.title_id join Salary on emp123.emp_no == Salary.emp_no join depart on 
    dept_employee.dept_no == depart.dept_no""")

In [73]:
df.printSchema()

root
 |-- emp_no: string (nullable = true)
 |-- emp_title_id: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- no_of_projects: integer (nullable = true)
 |-- last_performance_rating: string (nullable = true)
 |-- left_: string (nullable = true)
 |-- last_date: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- title: string (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)



In [74]:
df.show(5)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+---------+------+---------------+-------+--------------------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|last_performance_rating|left_|last_date|salary|          title|dept_no|           dept_name|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+---------+------+---------------+-------+--------------------+
| 40000|          Staff|   d002|           "Finance"|
| 53422|       Engineer|   d004|        "Production"|
| 48973|       Engineer|   d004|        "Production"|
| 40000|          Staff|   d003|   "Human Resources"|
| 40000|Senior Engineer|   d006|"Quality Management"|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+---------+------+---------------+-------+--------------------+
only showing top 5 rows



In [75]:
df.columns

['emp_no',
 'emp_title_id',
 'birth_date',
 'first_name',
 'last_name',
 'sex',
 'hire_date',
 'no_of_projects',
 'last_performance_rating',
 'left_',
 'last_date',
 'salary',
 'title',
 'dept_no',
 'dept_name']

In [76]:
#Creating the summary report of Countinous Variable
df.select(['no_of_projects','salary']).summary().show()

+-------+------------------+------------------+
|summary|    no_of_projects|            salary|
+-------+------------------+------------------+
|  count|            331603|            331603|
|   mean|5.5048567111877755| 52972.22543221865|
| stddev|2.8728285038833126|14299.003000805926|
|    min|                 1|             40000|
|    25%|                 3|             40000|
|    50%|                 6|             48688|
|    75%|                 8|             61761|
|    max|                10|            129492|
+-------+------------------+------------------+



# Data Preparation Encoding

In [77]:
df.show(10)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+----------+------+---------------+-------+--------------------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|last_performance_rating|left_| last_date|salary|          title|dept_no|           dept_name|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+----------+------+---------------+-------+--------------------+
| 40000|          Staff|   d002|           "Finance"|
| 53422|       Engineer|   d004|        "Production"|
| 48973|       Engineer|   d004|        "Production"|
| 40000|          Staff|   d003|   "Human Resources"|
| 40000|Senior Engineer|   d006|"Quality Management"|
| 40000|Senior Engineer|   d006|"Quality Management"|
| 56087|          Staff|   d003|   "Human Resources"|
| 40000|   Senior Staff|   d002|           "Finance"|
| 54816|       Engineer|   d006|"Qual

In [78]:
df.printSchema()

root
 |-- emp_no: string (nullable = true)
 |-- emp_title_id: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- no_of_projects: integer (nullable = true)
 |-- last_performance_rating: string (nullable = true)
 |-- left_: string (nullable = true)
 |-- last_date: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- title: string (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)



In [92]:
#Encoding all categorical features
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer,OneHotEncoderEstimator,StandardScaler

In [101]:
df_new=df

In [102]:
continious_features =['emp_no','emp_title_id','birth_date','first_name','last_name','hire_date',
                      'no_of_projects','last_date','title','dept_no']
categorical_features= ['sex','last_performance_rating','salary','dept_name']

StringIndexer

In [103]:
SI_sex = StringIndexer(inputCol='sex', outputCol='sex_index')
SI_last_performance_rating= StringIndexer(inputCol='last_performance_rating', outputCol='last_performance_rating_index')
SI_salary=StringIndexer(inputCol='salary', outputCol='salary_index')
SI_dept_name=StringIndexer(inputCol='dept_name', outputCol='dept_name_index')

df_new=SI_sex.fit(df_new).transform(df_new)
df_new=SI_last_performance_rating.fit(df_new).transform(df_new)
df_new=SI_salary.fit(df_new).transform(df_new)
df_new=SI_dept_name.fit(df_new).transform(df_new)

df_new.select('sex','sex_index','last_performance_rating','last_performance_rating_index','salary','salary_index','dept_name','dept_name_index').show(10)


+---+---------+-----------------------+-----------------------------+------+------------+--------------------+---------------+
|sex|sex_index|last_performance_rating|last_performance_rating_index|salary|salary_index|           dept_name|dept_name_index|
+---+---------+-----------------------+-----------------------------+------+------------+--------------------+---------------+
|  M|      0.0|                      A|                          1.0| 40000|         0.0|           "Finance"|            8.0|
|  F|      1.0|                      C|                          2.0| 53422|      4384.0|        "Production"|            1.0|
|  F|      1.0|                      A|                          1.0| 48973|     20860.0|        "Production"|            1.0|
|  M|      0.0|                      C|                          2.0| 40000|         0.0|   "Human Resources"|            7.0|
|  F|      1.0|                      B|                          0.0| 40000|         0.0|"Quality Management"| 

One-Hot Encoder

In [104]:
#create object and specify input and output column
encoder= OneHotEncoderEstimator().setInputCols(['sex_index','last_performance_rating_index','salary_index','dept_name_index']).setOutputCols(['sex_vec','last_performance_rating_vec','salary_vec','dept_name_vec'])

In [105]:
df_new= encoder.fit(df_new).transform(df_new)

In [106]:
df_new.select('sex_vec','last_performance_rating_vec','salary_vec','dept_name_vec').show(5)

+-------------+---------------------------+--------------------+-------------+
|      sex_vec|last_performance_rating_vec|          salary_vec|dept_name_vec|
+-------------+---------------------------+--------------------+-------------+
|(1,[0],[1.0])|              (4,[1],[1.0])|   (50354,[0],[1.0])|    (8,[],[])|
|    (1,[],[])|              (4,[2],[1.0])|(50354,[4384],[1.0])|(8,[1],[1.0])|
|    (1,[],[])|              (4,[1],[1.0])|(50354,[20860],[1...|(8,[1],[1.0])|
|(1,[0],[1.0])|              (4,[2],[1.0])|   (50354,[0],[1.0])|(8,[7],[1.0])|
|    (1,[],[])|              (4,[0],[1.0])|   (50354,[0],[1.0])|(8,[6],[1.0])|
+-------------+---------------------------+--------------------+-------------+
only showing top 5 rows



In [107]:
#create vector from categorical columns
featureCols =['sex_vec','last_performance_rating_vec','salary_vec','dept_name_vec']

Vector Assembler

In [108]:
Vector_assembler = VectorAssembler(inputCols = featureCols,outputCol = "features")

In [109]:
caps_df= Vector_assembler.transform(df_new)

In [110]:
caps_df.columns

['emp_no',
 'emp_title_id',
 'birth_date',
 'first_name',
 'last_name',
 'sex',
 'hire_date',
 'no_of_projects',
 'last_performance_rating',
 'left_',
 'last_date',
 'salary',
 'title',
 'dept_no',
 'dept_name',
 'sex_index',
 'last_performance_rating_index',
 'salary_index',
 'dept_name_index',
 'sex_vec',
 'last_performance_rating_vec',
 'salary_vec',
 'dept_name_vec',
 'features']

Label Indexer

In [111]:
caps_df= caps_df.withColumn('label', when(caps_df.left_ == '1',1).otherwise(0))

In [112]:
caps_df.select('left_','label').show(5)

+-----+-----+
|left_|label|
+-----+-----+
|    0|    0|
|    0|    0|
|    0|    0|
|    0|    0|
|    0|    0|
+-----+-----+
only showing top 5 rows



In [113]:
caps_df.show(10)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+----------+------+---------------+-------+--------------------+---------+-----------------------------+------------+---------------+-------------+---------------------------+--------------------+-------------+--------------------+-----+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|last_performance_rating|left_| last_date|salary|          title|dept_no|           dept_name|sex_index|last_performance_rating_index|salary_index|dept_name_index|      sex_vec|last_performance_rating_vec|          salary_vec|dept_name_vec|            features|label|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+----------+------+---------------+-------+--------------------+---------+-----------------------------+------------+---------------+-------------+---------------------------+-------------

TRain & Test

In [114]:
train_df,test_df = caps_df.randomSplit([0.7,0.3], seed= 50)

In [115]:
#Build the Linear Regression Model
from pyspark.ml.classification import LogisticRegression 

In [116]:
logrg = LogisticRegression(featuresCol= 'features',labelCol='label',maxIter=5)

In [117]:
lr = logrg.fit(train_df)

In [118]:
lr.intercept

-2.341359511856782

In [119]:
lr.coefficients

DenseVector([-0.1696, -0.2028, -0.1973, -0.1901, -0.1727, 0.4882, 1.6534, -0.0127, 1.8514, -0.0993, 1.7741, 1.7518, 0.935, -1.615, 0.0113, -0.0258, 0.1007, 0.7013, 1.805, 1.4795, 1.1518, 0.7139, -0.1331, -0.1469, -1.6053, 0.0389, -0.0343, 1.461, -0.027, -1.6068, 1.0532, 1.1251, -1.6098, 1.1829, 1.6381, 0.3107, -1.6135, 1.2196, 0.0686, 1.6442, 1.2995, -0.0373, -1.6089, -1.6338, -0.3117, 1.7163, 0.2433, 1.4961, -1.6225, 0.054, -1.6158, 2.1418, -1.6133, -0.0182, -1.6081, 1.6281, 1.3702, 0.0229, 1.5841, -1.6088, 0.9506, -1.6076, -1.6271, 1.2413, -1.6236, 0.1024, 2.5168, 2.1188, -1.6237, 0.2119, 1.6, -1.6037, 0.3264, 0.0233, 0.2003, 1.4943, -1.6129, -1.6203, -1.6048, 0.7769, -1.6118, 2.2634, 0.3692, 0.4032, -1.604, -1.6064, 2.076, 0.2284, 2.1017, -1.6214, 0.0778, 1.1809, 0.2181, -1.5991, 2.1875, 1.825, -1.623, -1.6012, -1.595, 2.2985, 1.1931, 0.5433, -1.6247, 0.1113, 0.1424, 0.5102, 1.8298, -1.6137, 2.206, -0.1099, -1.6186, 1.6361, -1.6051, -1.6195, 0.9377, -1.6221, 1.1942, 0.2083, 0.2087, 

In [120]:
y_pred_test = lr.transform(test_df)

In [121]:
y_pred_train = lr.transform(train_df)

In [128]:
y_pred_train.select(['label',
 'rawPrediction',
 'probability',
 'prediction']).toPandas().head(10)

,label,rawPrediction,probability,prediction
0,0,"[4.2245770619225755, -4.2245770619225755]","[0.9855794726293349, 0.0144205273706652]",0.0
1,0,"[2.326917636895995, -2.326917636895995]","[0.9110819456756485, 0.08891805432435157]",0.0
2,0,"[2.157280982601104, -2.157280982601104]","[0.8963471997390388, 0.10365280026096123]",0.0
3,0,"[2.3229259951137684, -2.3229259951137684]","[0.9107580451971714, 0.08924195480282866]",0.0
4,0,"[4.67260139127028, -4.67260139127028]","[0.9907386540519525, 0.009261345948047627]",0.0
5,0,"[4.2258145332591655, -4.2258145332591655]","[0.9855970497203049, 0.01440295027969499]",0.0
6,0,"[4.691109527497286, -4.691109527497286]","[0.9909069434953706, 0.009093056504629425]",0.0
7,1,"[0.6990063207060457, -0.6990063207060457]","[0.6679674238692578, 0.3320325761307422]",0.0
8,0,"[0.5329767422355876, -0.5329767422355876]","[0.6301771218241043, 0.3698228781758956]",0.0
9,0,"[2.293643436812085, -2.293643436812085]","[0.9083492211251641, 0.09165077887483586]",0.0


In [123]:
evaluator = BinaryClassificationEvaluator()

In [124]:
evaluator.evaluate(y_pred_test)

0.5471213624667834

Accuracy

In [132]:
y_pred_test.filter(y_pred_test.label == y_pred_test.prediction).count() / float(y_pred_test.count())

0.8796217199481423

In [126]:
    print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial. (default: auto)
featuresCol: features column name (default: features, current: features)
fitIntercept: whether to fit an intercept term (default: True)
labelCol: label column name (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. (undefined)
maxIter: maximum number of iterations (>= 0) (default: 100, current: 5)
predictionCol: prediction column name (default: prediction)
probabilityCol: Column name for pr